<a href="https://colab.research.google.com/github/Artin200912/DL-/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm
from PIL import Image
import os

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


defining the dataset path

In [5]:
data_path = '/content/face-expression-recognition-dataset'
data_train = "/content/face-expression-recognition-dataset/images/train"
data_test = "/content/emotion-detection-fer/test"
data_validation = "/content/face-expression-recognition-dataset/images/validation"

In [15]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: artindaneshvar
Your Kaggle Key: ··········


100%|██████████| 121M/121M [00:00<00:00, 159MB/s]


In [16]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])

In [17]:
class CustomDataset(ImageFolder):
  def __init__(self, root_dir, transform=None):
    super(CustomDataset, self).__init__(root=root_dir, transform=transform)

In [18]:
def load_images(self):
  images = []
  for Images in self.classes:
    class_path = os.path.join(self.root_dir, Images)
    class_idx = self.class_to_idx[Images]
    for img_name in os.listdir(class_path):
      img_path = os.path.join(class_path, img_name)
      images.append((img_path, class_idx))
    return images

In [19]:
def __len__(self):
  return len(self.images)

In [20]:
def __Get_item__(self, idx):
  img_path, label = self.images[idx]
  image = Image.open(img_path).convert("L")
  if self.transform:
    image = self.transform(image)

  return image, label

In [21]:
train_dataset = CustomDataset(root_dir=data_train, transform=transform)
test_dataset = CustomDataset(root_dir=data_test, transform=transform)
validation_dataset = CustomDataset(root_dir=data_validation, transform=transform)


In [23]:
train_loader = train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False, num_workers=4)

In [24]:
class EmotionNN(nn.Module):
  def __init__(self):
    super(EmotionNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
    self.FullyConneted1 = nn.Linear(128 * 6 * 6, 512)
    self.FullyConneted2 = nn.Linear(512, 7)

  def forward(self, x):
    x = self.pool(nn.functional.relu(self.conv1(x)))
    x = self.pool(nn.functional.relu(self.conv2(x)))
    x = self.pool(nn.functional.relu(self.conv3(x)))
    x = x.view(-1, 128 * 6 * 6)
    x = nn.functional.relu(self.FullyConneted1(x))
    x = nn.functional.sigmoid(self.FullyConneted2(x))
    return x

In [25]:
model = EmotionNN()

In [26]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [28]:
num_of_epochs = 7
for epoch in range(num_of_epochs):
  model.train()
  running_loss = 0.0
  for images, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1} / {num_of_epochs}'):
    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_func(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

  print(f"Epoch {epoch + 1}/{num_of_epochs}, Loss: {running_loss / len(train_loader)}")
print("Training Loop finished successfully")


Epoch 1 / 7: 100%|██████████| 451/451 [02:16<00:00,  3.30it/s]


Epoch 1/7, Loss: 1.7888593179423635


Epoch 2 / 7: 100%|██████████| 451/451 [02:27<00:00,  3.05it/s]


Epoch 2/7, Loss: 1.727196387599683


Epoch 3 / 7: 100%|██████████| 451/451 [02:27<00:00,  3.05it/s]


...

Epoch 5/7, Loss: 1.6398299691946703


Epoch 6 / 7: 100%|██████████| 451/451 [02:35<00:00,  2.89it/s]


Epoch 6/7, Loss: 1.6182557109189932


Epoch 7 / 7: 100%|██████████| 451/451 [03:02<00:00,  2.47it/s]

Epoch 7/7, Loss: 1.5951401058691832
Training Loop finished successfully


In [29]:
model.eval()
correct = 0
total = 0

In [31]:
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

Testing: 100%|██████████| 113/113 [00:17<00:00,  6.63it/s]


In [32]:
accuracy = correct / total
print(f'Accuracy on test set{accuracy * 100:.2f}%')


Accuracy on test set52.00%


In [33]:
torch.save(model.state_dict(), 'project_model.pth')

In [34]:
model = EmotionNN()
model.load_state_dict(torch.load('project_model.pth'))
model.eval()

EmotionNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (FullyConneted1): Linear(in_features=4608, out_features=512, bias=True)
  (FullyConneted2): Linear(in_features=512, out_features=7, bias=True)
)

In [36]:
predictions = []

with torch.no_grad():
    for inputs, _ in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.numpy())

In [43]:
input_image = Image.open('/content/smile.jpg')
input_tensor = transform(input_image).unsqueeze(0)
emotion_mapping = {0: 'Angry', 1: 'Happy', 2: 'Sad', 3: 'Surprised', 4: 'Neutral'}

In [44]:
with torch.no_grad():
    model.eval()
    output = model(input_tensor)
    _, predicted_label = torch.max(output, 1)

print(f'Predicted emotion is {predicted_label.item()}')


Predicted emotion is 2
